<a href="https://colab.research.google.com/github/elliot-brooks/nlu-coursework/blob/main/src/av_model_two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!pip install -U transformers
!pip install -U accelerate

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertModel
import torch
import re

# Load training data

In [3]:
training_corpus = pd.read_csv("train.csv", encoding='utf-8')

# Pre-process training data

In [4]:
def preprocess(string):
  output = str(string).lower()
  separated_string = re.sub(r'([^\w\s])', r' \1 ', str(string))
  return output

# Prepare data for Distilled Bert
def prepare_data(data) :
  data["text_1"] = data["text_1"].apply(lambda x: preprocess(x))
  data["text_2"] = data["text_2"].apply(lambda x: preprocess(x))
  concat_pairs = []
  for index, row in data.iterrows():
      concatenated_pair = row["text_1"] + " [SEP] " + row["text_2"]
      concat_pairs.append(concatenated_pair)
  return concat_pairs

concat_data = prepare_data(training_corpus)

Load BERT Models

In [5]:
tokeniser = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
bert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected i

Create BERT Embeddings

In [18]:
SEQ_LENGTH = 128
BATCH_SIZE = 32

def create_bert_embeddings_batch(texts, tokeniser, model, batch_size, seq_length) :
  embeddings = []
  for i in range(0, len(texts), batch_size) :
    batch = texts[i:i + batch_size]
    inputs = tokeniser.batch_encode_plus(batch, padding='max_length', truncation=True, return_tensors='tf', max_length=seq_length)

    # Create embeddings
    outputs = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])

    last_hidden_state_CLS = outputs.last_hidden_state[:, 0, :]

    embeddings.append(last_hidden_state_CLS)
  return embeddings

bert_embeddings = create_bert_embeddings_batch(concat_data, tokeniser, bert_model, BATCH_SIZE, SEQ_LENGTH)
train_labels = np.array(training_corpus['label'])

# Define Language Model

In [23]:
BERT_ENCODING_DIM = 768

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(BERT_ENCODING_DIM,)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 128)               98432     
                                                                 
 dropout_25 (Dropout)        (None, 128)               0         
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 dropout_26 (Dropout)        (None, 64)                0         
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                                                 
Total params: 106753 (417.00 KB)
Trainable params: 106753 (417.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Train Model

In [30]:
num_samples = train_labels.shape[0]



# Concatenate BERT batches
train_inputs = np.concatenate(bert_embeddings, axis=0)
print(train_inputs.shape)

model.fit(train_inputs, train_labels, epochs=10, batch_size=8)

(30000, 768)
Epoch 1/10
3750/3750 [==============================] - 15s 4ms/step - loss: 0.6905 - accuracy: 0.5082
Epoch 2/10
3750/3750 [==============================] - 15s 4ms/step - loss: 0.6906 - accuracy: 0.5066
Epoch 3/10
3750/3750 [==============================] - 14s 4ms/step - loss: 0.6901 - accuracy: 0.5055
Epoch 4/10
3750/3750 [==============================] - 14s 4ms/step - loss: 0.6906 - accuracy: 0.5075
Epoch 5/10
3750/3750 [==============================] - 13s 4ms/step - loss: 0.6902 - accuracy: 0.5068
Epoch 6/10
3750/3750 [==============================] - 14s 4ms/step - loss: 0.6899 - accuracy: 0.5061
Epoch 7/10
3750/3750 [==============================] - 13s 4ms/step - loss: 0.6904 - accuracy: 0.5076
Epoch 8/10
3750/3750 [==============================] - 13s 4ms/step - loss: 0.6903 - accuracy: 0.5058
Epoch 9/10
3750/3750 [==============================] - 13s 4ms/step - loss: 0.6894 - accuracy: 0.5070
Epoch 10/10
3750/3750 [==============================] - 13s

# Save Model

In [31]:
model.save("AV_SVM_MODEL")

In [32]:
!zip -r /content/SVM_MODEL.zip /content/AV_SVM_MODEL

  adding: content/AV_SVM_MODEL/ (stored 0%)
  adding: content/AV_SVM_MODEL/fingerprint.pb (stored 0%)
  adding: content/AV_SVM_MODEL/keras_metadata.pb (deflated 87%)
  adding: content/AV_SVM_MODEL/saved_model.pb (deflated 87%)
  adding: content/AV_SVM_MODEL/variables/ (stored 0%)
  adding: content/AV_SVM_MODEL/variables/variables.data-00000-of-00001 (deflated 36%)
  adding: content/AV_SVM_MODEL/variables/variables.index (deflated 60%)
  adding: content/AV_SVM_MODEL/assets/ (stored 0%)
